In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import os

In [ ]:
# Define the atoms in the LiH molecule
symbols = ["Li", "H"]

# Define the coordinates (in Ångströms)
coordinates = np.array([
    [0.0, 0.0, 0.0],  # Lithium atom at the origin
    [0.0, 0.0, 1.6]   # Hydrogen atom positioned 1.6 Å along the z-axis
])

In [ ]:
# Define the LiH Hamiltonian and the number of qubits required
# Default STO-3G basis set
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0)

# 1 spatial orbital for H and 5 for Li
# Each spatial orbital is doubly-degenerate (2 spin orbitals)
print(f"{qubits} qubits required")

In [ ]:
# Which orbitals are occupied (1) or unoccupied (0) in the mean-field (Hartree-Fock) solution
electrons = 4  # 1 from H and 3 from Li
hf = qchem.hf_state(electrons=electrons, orbitals=qubits)  # Creates the Hartree-Fock state

# Show that the first 4 orbitals are occupied
# Print the state in bra-ket notation
print(f"Hartree-Fock state: {'|' + ''.join(str(bit) for bit in hf) + '⟩'}")

In [ ]:
# Define the number of required quantum wires / logical qubits
num_wires = qubits

# Create quantum device simulator backend
dev = qml.device("default.qubit",  # Noiseless state vector simulator
                 wires=num_wires)


@qml.qnode(dev)  # Transforms exp_energy below into a quantum node
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))

    # Return the expectation value of the molecular Hamiltonian
    return qml.expval(hamiltonian)

# Calculate ⟨ψ_hf| hamiltonian |ψ_hf⟩ in Hartree (Ha) units
# 1 Ha = 27.2 eV
exp_energy(hf)

In [ ]:
# Generate excitation indices
singles, doubles = excitations(electrons=electrons, orbitals=qubits)
print(f"Singles Excitations: {singles}")  # e.g: [0, 4] is a single excitation of an electron in orbital 0 to orbital 4
print(f"Doubles Excitations: {doubles}")  # e.g: [0, 1, 4, 5] is a double excitation of electrons in orbitals 0 and 1 to orbitals 4 and 5


# Define an ansatz using the excitations
def ansatz(params):
    # Initialize the qubit register in the Hartree-Fock state
    qml.BasisState(hf, wires=range(num_wires))

    # We only loop over the double excitations for LiH
    for i, excitation in enumerate(doubles):
        # Unitary quantum gate representing a double excitation
        qml.DoubleExcitation(params[i], wires=excitation)

In [ ]:
# Define the VQE cost function
@qml.qnode(dev)
def cost_function(params):
    ansatz(params)

    # Measure the expectation value of the Hamiltonian after applying the ansatz:
    # E(theta) = ⟨ψ(theta)| H |ψ(theta)⟩
    return qml.expval(hamiltonian)

# Create a vector of zeros with the same length as the number of double excitations
initial_params = np.zeros(len(doubles))

# Confirm the initial energy of the system using the Hartree-Fock state
# This is the starting point for classical optimization
cost_function(initial_params)

In [ ]:
# The optimizer uses automatic differentiation to compute gradients and adjust the parameters
opt = qml.GradientDescentOptimizer(stepsize=0.1)    # Initializes the gradient descent optimizer
theta = np.zeros(len(doubles), requires_grad=True)  # Initializes a vector of zeros as the starting parameters for the ansatz

# Initialize energy and parameter tracking lists
energy = [cost_function(theta)]
angle = [theta]

# VQE optimization loop
max_iterations = 50
for n in range(max_iterations):
    # Updates parameters theta to minimize the cost function (energy)
    theta, prev_energy = opt.step_and_cost(cost_function, theta)
    energy.append(cost_function(theta))
    angle.append(theta)

# Plotting VQE convergence
plt.plot(range(len(energy)), energy)
plt.xlabel('Iteration')
plt.ylabel('Energy (Ha)')
plt.title('VQE Convergence')

# Ensure the "images" directory exists
os.makedirs('images', exist_ok=True)

# Save the figure to the "images" directory
plt.savefig('images/LiH_convergence.png')
plt.show()

In [ ]:
print(f"Final ground state energy: {energy[-1]:.8f} Ha")

# Optimized angles in the DoubleExcitation gates for first two excitations
print(f"Final parameters: {theta[0]:.8f}, {theta[1]:.8f}")

In [ ]:
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)

    # Return the entire quantum statevector
    return qml.state()

# Run the circuit using the final optimized angles
final_state = ground_state(theta)

# Find non-zero (or near non-zero) entries
threshold = 1e-2  # Recommended smaller threshold to capture all significant amplitudes
non_zero_indices = np.where(np.abs(final_state) > threshold)[0]
non_zero_amplitudes = final_state[non_zero_indices]

# Build the full ket notation string
ket_terms = []
for idx, amp in zip(non_zero_indices, non_zero_amplitudes):
    binary_state = f"|{idx:0{num_wires}b}⟩"
    
    # Format the amplitude nicely
    amp_str = f"{amp.real:.4f}" if np.isclose(amp.imag, 0, atol=1e-6) else f"({amp.real:.4f} + {amp.imag:.4f}j)"
    ket_terms.append(f"{amp_str}{binary_state}")

# Join all terms into one quantum state expression and print
ket_notation = " + ".join(ket_terms)
print(f"Ground state of LiH:\n|ψ⟩ = {ket_notation}")

In [ ]:
# Prepare labels and amplitudes for the plot
labels = [f"|{idx}⟩" for idx in non_zero_indices]
amplitudes = np.abs(non_zero_amplitudes)

# Bar plot
plt.figure(figsize=(12, 6))
plt.bar(labels, amplitudes)
plt.xlabel('Basis States', fontsize=14)
plt.ylabel('Amplitude', fontsize=14)
plt.title('Ground State of LiH (VQE Result)', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Save the figure to the "images" directory
plt.savefig('images/LiH_ground_state.png')
plt.show()